In [1]:
import cv2
import numpy as np

In [2]:
def cv_show(img, name='image'):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [3]:
# 拿編號 99這張照片來測試
path = r'C:\Users\jason\Desktop\python\01_project\09_esun_2021\test'
img = cv2.imread(f'{path}\99.jpg')
cv_show(img)

In [342]:
cv_show(img)

In [392]:

out_w = 128
out_h = 128
top_size = int((128 - img.shape[0]) / 2)
bottom_size = int((128 - img.shape[0]) / 2)
dif_1 = 128 - (top_size + bottom_size + img.shape[0])
top_size += dif_1
left_size = int((128 - img.shape[1]) / 2)
right_size = int((128 - img.shape[1]) / 2)
dif_2 = 128 - (left_size + right_size + img.shape[1])
left_size += dif_2    
padding_v = tuple(img[-1, -1, :])
if img.shape[1] <= 128:
    constant = cv2.copyMakeBorder(img, 
                              top_size, bottom_size, left_size, right_size, 
                              borderType=cv2.BORDER_CONSTANT, 
                              value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
else:
    constant = cv2.copyMakeBorder(img, 
                      top_size, bottom_size, 0, 0, 
                      borderType=cv2.BORDER_CONSTANT, 
                      value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
    constant = cv2.resize(constant, (128, 128))
        


In [393]:
# 灰階 二值化
gray = cv2.cvtColor(constant, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_OTSU)
# 黑白轉換
thresh = 255 - thresh
cv_show(thresh)

In [394]:
# 閉操作
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
cv_show(closed)
# 腐蝕 & 膨脹
closed = cv2.erode(closed, None, iterations=1)
closed = cv2.dilate(closed, None, iterations=1)
cv_show(closed)

In [395]:
# 輪廓檢測
contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print(len(contours))

2


In [396]:
draw_img = constant.copy()
res = cv2.drawContours(draw_img, contours, -1, (0, 0, 255), 2) # 繪製圖像, 輪廓, 輪廓索引, 顏色模式(B, G, R), 線條厚度
cv_show(res)

In [397]:
draw_img = constant.copy()
for c in contours:
    rect = cv2.minAreaRect(c)
    box = np.int0(cv2.boxPoints(rect))
    res = cv2.drawContours(draw_img, [box], -1, (0, 255, 0), 3)
cv_show(res)

In [398]:
# 取出最大的
c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))

In [399]:
cv2.contourArea(c)

1466.0

In [400]:
# 裁切
Xs = [i[0] for i in box]
Ys = [i[1] for i in box]
x1 = min(Xs)
x2 = max(Xs)
y1 = min(Ys)
y2 = max(Ys)
hight = y2 - y1
width = x2 - x1
cropImg = constant[y1: y1+hight, x1: x1+width]

In [401]:
cv_show(cropImg)

### function

In [336]:
def preprocessing_img(img):
    # 先padding再裁切
    
    out_w = 128
    out_h = 128
    top_size = int((128 - img.shape[0]) / 2)
    bottom_size = int((128 - img.shape[0]) / 2)
    dif_1 = 128 - (top_size + bottom_size + img.shape[0])
    top_size += dif_1
    left_size = int((128 - img.shape[1]) / 2)
    right_size = int((128 - img.shape[1]) / 2)
    dif_2 = 128 - (left_size + right_size + img.shape[1])
    left_size += dif_2    
    padding_v = tuple(img[-1, -1, :])
    if img.shape[1] <= 128:
        padding = cv2.copyMakeBorder(img, 
                                  top_size, bottom_size, left_size, right_size, 
                                  borderType=cv2.BORDER_CONSTANT, 
                                  value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
    else:
        padding = cv2.copyMakeBorder(img, 
                          top_size, bottom_size, 0, 0, 
                          borderType=cv2.BORDER_CONSTANT, 
                          value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
        padding = cv2.resize(constant, (128, 128))


    # 灰階 二值化
    gray = cv2.cvtColor(padding, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_OTSU)
    # 黑白轉換
    thresh = 255 - thresh
    # 閉操作
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    # 腐蝕 & 膨脹
    closed = cv2.erode(closed, None, iterations=1)
    closed = cv2.dilate(closed, None, iterations=1)
    # 輪廓檢測
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    if len(contours) == 0:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
        thresh = 255 - thresh

    else:    
        # 取出最大的
        c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
        if cv2.contourArea(c) > 1000:
            # 閉操作
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
            # 腐蝕 & 膨脹
            closed = cv2.erode(closed, None, iterations=1)
            closed = cv2.dilate(closed, None, iterations=1)
            # 輪廓檢測
            contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
            
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))
        # 裁切
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        hight = y2 - y1
        width = x2 - x1
        padding = padding[y1: y1+hight, x1: x1+width]
        thresh = thresh[y1: y1+hight, x1: x1+width]
    # reshape
    img2 = cv2.resize(thresh, (64, 64))
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
     
    return padding, img2

In [306]:
168 39384.jpg
246 25877.jpg
563 5620.jpg
594 1925.jpg
595 58299.jpg

SyntaxError: invalid syntax (<ipython-input-306-e58bf9ec193c>, line 1)

In [390]:
path = r'C:\Users\jason\Desktop\python\01_project\09_esun_2021\data-clean'
fold = 741    
idx = 10297
img = cv2.imread(f'{path}/{fold}/{idx}.jpg')
cv_show(img)

In [391]:
a, b = preprocessing_img(img)
cv_show(a)
cv_show(b)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-q0nmoxxv\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [190]:
img2.shape

(64, 64, 3)